In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, ArrayType, BooleanType, MapType
from pyspark.sql.functions import from_json, col, explode, count, when

In [0]:
df= spark.read.load("/mnt/customertransaction/raw/fivetran/mongo_trans/accounts")

In [0]:
data_schema = \
    StructType(
        fields=[
            StructField("account_id", IntegerType(), True),
            StructField("limit", IntegerType(), True),
            StructField("products", ArrayType(StringType()), True)
        ]
    )

In [0]:
df_defined = df.withColumn("data", from_json("data", data_schema))

In [0]:
df_final = \
    df_defined\
        .select(
            col("_id").alias("id"),
            col("data.account_id").alias("account_id"),
            col("data.limit").alias("limit"),
            col("data.products").alias("products"),
            col("_fivetran_synced").alias("ingest_date")
        ).dropDuplicates(["account_id"])

In [0]:
df_final\
    .write\
    .mode("overwrite")\
    .format("delta")\
    .saveAsTable("processed.ct.account")